In [1]:
#coding:utf-8
import torch
from itertools import product
import numpy as np

In [5]:
def generate_data():
    # 本函数生成0-9，10个数字的图片矩阵
    image_data=[]
    num_0 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,1,0,0,1,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_0)
    num_1 = torch.tensor(
    [[0,0,0,1,0,0],
    [0,0,1,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,1,1,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_1)
    num_2 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,0,1,0,0],
    [0,0,1,0,0,0],
    [0,1,1,1,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_2)
    num_3 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,0,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_3)
    num_4 = torch.tensor(
    [
    [0,0,0,0,1,0],
    [0,0,0,1,1,0],
    [0,0,1,0,1,0],
    [0,1,1,1,1,1],
    [0,0,0,0,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_4)
    num_5 = torch.tensor(
    [
    [0,1,1,1,0,0],
    [0,1,0,0,0,0],
    [0,1,1,1,0,0],
    [0,0,0,0,1,0],
    [0,1,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_5)
    num_6 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,1,0,0,0,0],
    [0,1,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_6)
    num_7 = torch.tensor(
    [
    [0,1,1,1,1,0],
    [0,0,0,0,1,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_7)
    num_8 = torch.tensor(
    [[0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,1,0,0,1,0],
    [0,0,1,1,0,0],
    [0,0,0,0,0,0]])
    image_data.append(num_8)
    num_9 = torch.tensor(
    [[0,0,1,1,1,0],
    [0,1,0,0,1,0],
    [0,0,1,1,1,0],
    [0,1,0,0,1,0],
    [0,0,0,0,1,0],
    [0,0,0,0,0,0]])
    image_data.append(num_9)
    image_label=[0,1,2,3,4,5,6,7,8,9]
    image_label = torch.from_numpy(np.array(image_label)).view(10,1)
    return image_data,image_label
    
def get_feature(x):
    #feature=[0,0,0,0]
    # 下面添加提取图像x的特征feature的代码
    def get_shadow(x,dim):
        feature  =torch.sum(x,dim)
        feature = feature.float()
        # 归一化
        for i in range(0,feature.shape[0]):
            feature[i]=feature[i]/sum(feature)

        feature = feature.view(1,6)
        return feature
    feature  = get_shadow(x,0)
    
    return feature

def get_all_features(image_data):
    num = len(image_data) #样本个数
    features = torch.zeros(num, 6)
    for i in range(num):       
        features[i,:] = get_feature(image_data[i])
    return features

def model(feature,weights):
    y=-1
    feature = torch.cat((feature,torch.tensor(1.0).view(1,1)),1)
    y = feature.mm(weights)
    return y

def get_all_preds(image_data, weights):
    """
    对所有的图片数据进行推理预测
    """
    features = get_all_features(image_data)
    preds = torch.zeros(len(image_data), 1)
    for i in range(len(image_data)):
        preds[i,0] = model(features[i,:].unsqueeze(0), weights).squeeze(0)
    return preds

def train_model(image_data,image_label,weights):
    # 学习率
    lr_list = [0.12, 0.08, 0.06, 0.04, 0.02,0.01, 0.005]    
    loss_fun = torch.nn.MSELoss()
    for epoch in range(0,2000): 
        lr=lr_list[int(epoch/500%len(lr_list))]
        #optimizer.zero_grad()
        preds = get_all_preds(image_data, weights)
        
        loss = loss_fun(image_label, preds)     
        loss.backward()
        #optimizer.step()
        weights.data = weights.data - weights.grad.data * lr
        weights.grad.data = torch.zeros(7,1)
        print("epoch=%s,loss=%s,weights=%s"%(epoch,loss,weights.view(7)))
        #loss=0
    return weights

## 训练基本情况
通过每次对所有图像前向计算之后，计算MSE来作为loss  
尝试了不同的学习率,不同的epoch（最大设到20000，但是基本从1000左右就降到5.5左右开始几乎不下降）

In [6]:
weights = torch.rand(7,1, requires_grad = True)
image_data,image_label = generate_data()
# 对模型进行训练：
weights=train_model(image_data,image_label,weights)

epoch=0,loss=tensor(14.2627, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 0.4882, 0.9691, 1.1695, 1.0195, 0.8873, 1.4439],
       grad_fn=<ViewBackward>)
epoch=1,loss=tensor(10.2329, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 0.5600, 1.0615, 1.3224, 1.2481, 0.8964, 1.7747],
       grad_fn=<ViewBackward>)
epoch=2,loss=tensor(8.9331, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 0.6065, 1.1107, 1.4005, 1.3848, 0.8995, 1.9621],
       grad_fn=<ViewBackward>)
epoch=3,loss=tensor(8.5104, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 0.6386, 1.1356, 1.4364, 1.4692, 0.8993, 2.0682],
       grad_fn=<ViewBackward>)
epoch=4,loss=tensor(8.3696, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 0.6627, 1.1466, 1.4485, 1.5240, 0.8970, 2.1283],
       grad_fn=<ViewBackward>)
epoch=5,loss=tensor(8.3193, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 0.6821, 1.1499, 1.4471, 1.5619, 0.8936, 2.1622],
       grad_fn=<ViewBackward>)
epoch=6,loss=tensor(8.2983, grad_fn=<MeanBackward0>),wei

epoch=53,loss=tensor(8.0496, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.2909, 0.8700, 0.7528, 2.1926, 0.6199, 2.1904],
       grad_fn=<ViewBackward>)
epoch=54,loss=tensor(8.0460, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.3025, 0.8647, 0.7414, 2.2023, 0.6135, 2.1900],
       grad_fn=<ViewBackward>)
epoch=55,loss=tensor(8.0424, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.3141, 0.8593, 0.7301, 2.2119, 0.6071, 2.1896],
       grad_fn=<ViewBackward>)
epoch=56,loss=tensor(8.0389, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.3257, 0.8540, 0.7190, 2.2214, 0.6007, 2.1892],
       grad_fn=<ViewBackward>)
epoch=57,loss=tensor(8.0354, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.3372, 0.8487, 0.7079, 2.2308, 0.5943, 2.1888],
       grad_fn=<ViewBackward>)
epoch=58,loss=tensor(8.0320, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.3488, 0.8434, 0.6970, 2.2401, 0.5878, 2.1883],
       grad_fn=<ViewBackward>)
epoch=59,loss=tensor(8.0286, grad_fn=<MeanBackward0>

       grad_fn=<ViewBackward>)
epoch=111,loss=tensor(7.8954, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.9107, 0.5876, 0.2518, 2.6280, 0.2377, 2.1653],
       grad_fn=<ViewBackward>)
epoch=112,loss=tensor(7.8935, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.9205, 0.5831, 0.2455, 2.6337, 0.2311, 2.1649],
       grad_fn=<ViewBackward>)
epoch=113,loss=tensor(7.8916, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.9302, 0.5787, 0.2392, 2.6393, 0.2245, 2.1644],
       grad_fn=<ViewBackward>)
epoch=114,loss=tensor(7.8897, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.9398, 0.5742, 0.2330, 2.6449, 0.2179, 2.1640],
       grad_fn=<ViewBackward>)
epoch=115,loss=tensor(7.8878, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.9495, 0.5698, 0.2269, 2.6504, 0.2113, 2.1636],
       grad_fn=<ViewBackward>)
epoch=116,loss=tensor(7.8859, grad_fn=<MeanBackward0>),weights=tensor([0.8579, 1.9591, 0.5654, 0.2208, 2.6559, 0.2047, 2.1631],
       grad_fn=<ViewBackward>)
epoch=117,loss=

       grad_fn=<ViewBackward>)
epoch=167,loss=tensor(7.8086, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.4108,  0.3494, -0.0216,  2.8852, -0.1198,  2.1420],
       grad_fn=<ViewBackward>)
epoch=168,loss=tensor(7.8074, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.4189,  0.3453, -0.0252,  2.8888, -0.1259,  2.1417],
       grad_fn=<ViewBackward>)
epoch=169,loss=tensor(7.8062, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.4270,  0.3412, -0.0289,  2.8925, -0.1319,  2.1413],
       grad_fn=<ViewBackward>)
epoch=170,loss=tensor(7.8050, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.4351,  0.3371, -0.0325,  2.8961, -0.1380,  2.1409],
       grad_fn=<ViewBackward>)
epoch=171,loss=tensor(7.8038, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.4432,  0.3331, -0.0360,  2.8997, -0.1440,  2.1405],
       grad_fn=<ViewBackward>)
epoch=172,loss=tensor(7.8026, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.4512,  0.3290, -0.0395,  2.9032, -0.1501,  2.1401],
    

       grad_fn=<ViewBackward>)
epoch=222,loss=tensor(7.7521, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.8221,  0.1304, -0.1784,  3.0535, -0.4347,  2.1228],
       grad_fn=<ViewBackward>)
epoch=223,loss=tensor(7.7513, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.8290,  0.1265, -0.1806,  3.0560, -0.4401,  2.1225],
       grad_fn=<ViewBackward>)
epoch=224,loss=tensor(7.7504, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.8358,  0.1226, -0.1827,  3.0586, -0.4454,  2.1222],
       grad_fn=<ViewBackward>)
epoch=225,loss=tensor(7.7496, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.8426,  0.1187, -0.1848,  3.0611, -0.4507,  2.1219],
       grad_fn=<ViewBackward>)
epoch=226,loss=tensor(7.7487, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.8493,  0.1148, -0.1869,  3.0636, -0.4560,  2.1215],
       grad_fn=<ViewBackward>)
epoch=227,loss=tensor(7.7479, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  2.8561,  0.1110, -0.1890,  3.0661, -0.4613,  2.1212],
    

...

       grad_fn=<ViewBackward>)
epoch=1733,loss=tensor(7.5688, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2546, -2.5229, -0.1188,  3.5923, -1.9823,  2.1812],
       grad_fn=<ViewBackward>)
epoch=1734,loss=tensor(7.5688, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2549, -2.5235, -0.1187,  3.5923, -1.9824,  2.1812],
       grad_fn=<ViewBackward>)
epoch=1735,loss=tensor(7.5688, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2553, -2.5241, -0.1186,  3.5923, -1.9824,  2.1813],
       grad_fn=<ViewBackward>)
epoch=1736,loss=tensor(7.5688, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2556, -2.5247, -0.1185,  3.5923, -1.9825,  2.1814],
       grad_fn=<ViewBackward>)
epoch=1737,loss=tensor(7.5688, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2559, -2.5253, -0.1184,  3.5923, -1.9825,  2.1814],
       grad_fn=<ViewBackward>)
epoch=1738,loss=tensor(7.5688, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2562, -2.5258, -0.1182,  3.5923, -1.9826,  2.1815]

       grad_fn=<ViewBackward>)
epoch=1782,loss=tensor(7.5683, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2703, -2.5515, -0.1131,  3.5920, -1.9852,  2.1844],
       grad_fn=<ViewBackward>)
epoch=1783,loss=tensor(7.5682, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2706, -2.5520, -0.1130,  3.5920, -1.9852,  2.1844],
       grad_fn=<ViewBackward>)
epoch=1784,loss=tensor(7.5682, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2709, -2.5526, -0.1129,  3.5920, -1.9853,  2.1845],
       grad_fn=<ViewBackward>)
epoch=1785,loss=tensor(7.5682, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2713, -2.5532, -0.1128,  3.5920, -1.9853,  2.1845],
       grad_fn=<ViewBackward>)
epoch=1786,loss=tensor(7.5682, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2716, -2.5538, -0.1127,  3.5920, -1.9854,  2.1846],
       grad_fn=<ViewBackward>)
epoch=1787,loss=tensor(7.5682, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2719, -2.5543, -0.1126,  3.5920, -1.9854,  2.1847]

       grad_fn=<ViewBackward>)
epoch=1839,loss=tensor(7.5676, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2881, -2.5841, -0.1066,  3.5916, -1.9882,  2.1880],
       grad_fn=<ViewBackward>)
epoch=1840,loss=tensor(7.5676, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2884, -2.5846, -0.1065,  3.5916, -1.9883,  2.1881],
       grad_fn=<ViewBackward>)
epoch=1841,loss=tensor(7.5676, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2888, -2.5852, -0.1064,  3.5916, -1.9883,  2.1882],
       grad_fn=<ViewBackward>)
epoch=1842,loss=tensor(7.5676, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2891, -2.5858, -0.1063,  3.5916, -1.9884,  2.1882],
       grad_fn=<ViewBackward>)
epoch=1843,loss=tensor(7.5676, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2894, -2.5863, -0.1062,  3.5915, -1.9884,  2.1883],
       grad_fn=<ViewBackward>)
epoch=1844,loss=tensor(7.5676, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.2897, -2.5869, -0.1061,  3.5915, -1.9885,  2.1884]

       grad_fn=<ViewBackward>)
epoch=1894,loss=tensor(7.5670, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3049, -2.6149, -0.1005,  3.5910, -1.9910,  2.1916],
       grad_fn=<ViewBackward>)
epoch=1895,loss=tensor(7.5670, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3052, -2.6155, -0.1004,  3.5910, -1.9910,  2.1917],
       grad_fn=<ViewBackward>)
epoch=1896,loss=tensor(7.5670, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3055, -2.6160, -0.1003,  3.5910, -1.9911,  2.1917],
       grad_fn=<ViewBackward>)
epoch=1897,loss=tensor(7.5670, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3058, -2.6166, -0.1001,  3.5910, -1.9911,  2.1918],
       grad_fn=<ViewBackward>)
epoch=1898,loss=tensor(7.5670, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3061, -2.6171, -0.1000,  3.5910, -1.9912,  2.1919],
       grad_fn=<ViewBackward>)
epoch=1899,loss=tensor(7.5670, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3064, -2.6177, -0.0999,  3.5910, -1.9912,  2.1919]

       grad_fn=<ViewBackward>)
epoch=1949,loss=tensor(7.5665, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3213, -2.6452, -0.0944,  3.5904, -1.9935,  2.1952],
       grad_fn=<ViewBackward>)
epoch=1950,loss=tensor(7.5665, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3216, -2.6457, -0.0943,  3.5904, -1.9935,  2.1952],
       grad_fn=<ViewBackward>)
epoch=1951,loss=tensor(7.5664, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3219, -2.6463, -0.0942,  3.5904, -1.9936,  2.1953],
       grad_fn=<ViewBackward>)
epoch=1952,loss=tensor(7.5664, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3222, -2.6468, -0.0941,  3.5904, -1.9936,  2.1954],
       grad_fn=<ViewBackward>)
epoch=1953,loss=tensor(7.5664, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3225, -2.6474, -0.0940,  3.5904, -1.9937,  2.1954],
       grad_fn=<ViewBackward>)
epoch=1954,loss=tensor(7.5664, grad_fn=<MeanBackward0>),weights=tensor([ 0.8579,  5.3228, -2.6479, -0.0939,  3.5904, -1.9937,  2.1955]

In [7]:
#对每张图片进行识别
print("对每张图片进行识别")
for i in range(0,10):
    x=image_data[i]
    #对当前图片提取特征
    feature=get_feature(x)
    # 对提取到得特征进行分类
    y = model(feature,weights)
    #打印出分类结果
    
    print("图像数字:{},识别结果:{},对应分类:{}, 是否识别正确:{}".format(i, y, y.round(), i==y.round()))
    #print("图像[%s]得分类结果是:[%s],它得特征是[%s]"%(i,y,feature))

对每张图片进行识别
图像数字:0,识别结果:tensor([[5.7735]], grad_fn=<MmBackward>),对应分类:tensor([[6.]], grad_fn=<RoundBackward>), 是否识别正确:tensor([[False]])
图像数字:1,识别结果:tensor([[3.2103]], grad_fn=<MmBackward>),对应分类:tensor([[3.]], grad_fn=<RoundBackward>), 是否识别正确:tensor([[False]])
图像数字:2,识别结果:tensor([[4.5523]], grad_fn=<MmBackward>),对应分类:tensor([[5.]], grad_fn=<RoundBackward>), 是否识别正确:tensor([[False]])
图像数字:3,识别结果:tensor([[3.5944]], grad_fn=<MmBackward>),对应分类:tensor([[4.]], grad_fn=<RoundBackward>), 是否识别正确:tensor([[False]])
图像数字:4,识别结果:tensor([[4.0117]], grad_fn=<MmBackward>),对应分类:tensor([[4.]], grad_fn=<RoundBackward>), 是否识别正确:tensor([[True]])
图像数字:5,识别结果:tensor([[4.4899]], grad_fn=<MmBackward>),对应分类:tensor([[4.]], grad_fn=<RoundBackward>), 是否识别正确:tensor([[False]])
图像数字:6,识别结果:tensor([[4.1733]], grad_fn=<MmBackward>),对应分类:tensor([[4.]], grad_fn=<RoundBackward>), 是否识别正确:tensor([[False]])
图像数字:7,识别结果:tensor([[4.9060]], grad_fn=<MmBackward>),对应分类:tensor([[5.]], grad_fn=<RoundBackward>), 是否识别正确:tensor([[False]])